In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputClassifier

In [2]:
df = pd.read_csv("shopping_behavior_updated.csv")
# df = pd.read_csv("data_behavior_expand_x30.csv")

In [3]:
df.head(3)

,Age,Customer ID,Item Purchased,Gender,Purchase Amount (USD),Location,Size,Category,Color,Season,Review Rating,Subscription Status,Shipping Type,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,55,1,Blouse,Male,53.0,Kentucky,L,Clothing,Gray,Winter,3.1,Yes,Express,Yes,14.0,Venmo,Fortnightly
1,19,2,Sweater,Male,64.0,Maine,L,Clothing,Maroon,Winter,3.1,Yes,Express,Yes,2.0,Cash,Fortnightly
2,50,3,Jeans,Male,73.0,Massachusetts,S,Clothing,Maroon,Spring,3.1,Yes,Free Shipping,Yes,23.0,Credit Card,Weekly


In [4]:
df = df.drop(labels=["Customer ID","Review Rating","Shipping Type","Promo Code Used"], axis = 1)
df.head(3)

,Age,Item Purchased,Gender,Purchase Amount (USD),Location,Size,Category,Color,Season,Subscription Status,Previous Purchases,Payment Method,Frequency of Purchases
0,55,Blouse,Male,53.0,Kentucky,L,Clothing,Gray,Winter,Yes,14.0,Venmo,Fortnightly
1,19,Sweater,Male,64.0,Maine,L,Clothing,Maroon,Winter,Yes,2.0,Cash,Fortnightly
2,50,Jeans,Male,73.0,Massachusetts,S,Clothing,Maroon,Spring,Yes,23.0,Credit Card,Weekly


In [5]:
print(df['Category'].unique())
print("*"*70)
print(df['Location'].unique())
print("*"*70)
print(df['Color'].unique())
print("*"*70)
print(df['Season'].unique())
print("*"*70)
print(df['Gender'].unique())
print("*"*70)
print(df['Subscription Status'].unique())
print("*"*70)
print(df['Frequency of Purchases'].unique())
print("*"*70)
print(df['Item Purchased'].unique())


['Clothing' 'Footwear' 'Outerwear' 'Accessories']
**********************************************************************
['Kentucky' 'Maine' 'Massachusetts' 'Rhode Island' 'Oregon' 'Wyoming'
 'Montana' 'Louisiana' 'West Virginia' 'Missouri' 'Arkansas' 'Hawaii'
 'Delaware' 'New Hampshire' 'New York' 'Alabama' 'Mississippi'
 'North Carolina' 'California' 'Oklahoma' 'Florida' 'Texas' 'Nevada'
 'Kansas' 'Colorado' 'North Dakota' 'Illinois' 'Indiana' 'Arizona'
 'Alaska' 'Tennessee' 'Ohio' 'New Jersey' 'Maryland' 'Vermont'
 'New Mexico' 'South Carolina' 'Idaho' 'Pennsylvania' 'Connecticut' 'Utah'
 'Virginia' 'Georgia' 'Nebraska' 'Iowa' 'South Dakota' 'Minnesota'
 'Washington' 'Wisconsin' 'Michigan']
**********************************************************************
['Gray' 'Maroon' 'Turquoise' 'White' 'Charcoal' 'Silver' 'Pink' 'Purple'
 'Olive' 'Gold' 'Violet' 'Teal' 'Lavender' 'Black' 'Green' 'Peach' 'Red'
 'Cyan' 'Brown' 'Beige' 'Orange' 'Indigo' 'Yellow' 'Magenta' 'Blue']
***********

In [6]:
names = df.columns
types = df.dtypes

scaler = MinMaxScaler()

label_encoders = {}
for col in names:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


# Dự Đoán Xu Hướng Thời Trang

In [7]:
X_trend = df[['Age', 'Gender', 'Category', 'Location', 'Color', 'Season']]
Y_trend = df['Item Purchased']
X_trend_scaled = scaler.fit_transform(X_trend)
model_trend = RandomForestClassifier(random_state=42)
model_trend.fit(X_trend_scaled, Y_trend)
def predict_trend_item(age, gender, category, location, color, season):
  new_customer_data = pd.DataFrame({
    'Age': [age],
    'Gender': [gender],
    'Category': [category],
    'Location': [location],
    'Color': [color],
    'Season': [season]
  })
  # Mã hóa dữ liệu mới
  for col in ['Age','Gender', 'Category', 'Location', 'Color', 'Season']:
      new_customer_data[col] = label_encoders[col].transform(new_customer_data[col])

  # Đảm bảo rằng new_customer_data chỉ chứa các cột cần thiết
  new_customer_data = new_customer_data[['Age', 'Gender', 'Category', 'Location', 'Color', 'Season']]

  # Chuẩn hóa dữ liệu mới
  new_customer_data_scaled = scaler.transform(new_customer_data)

  # Dự đoán sản phẩm
  predicted_item = model_trend.predict(new_customer_data_scaled)

  # Chuyển đổi dự đoán về tên sản phẩm gốc
  predicted_item_name = label_encoders['Item Purchased'].inverse_transform(predicted_item)

  # Hiển thị kết quả dự đoán
  print("Dự đoán sản phẩm cho khách hàng:", predicted_item_name)

In [8]:
predict_trend_item(30, 'Male', 'Clothing', 'Kentucky', 'Black', 'Spring')

Dự đoán sản phẩm cho khách hàng: ['Skirt']


# Dự đoán phân khúc khách hàng

In [9]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [10]:
# Chọn các cột để phân cụm
X_class = df[['Age', 'Gender', 'Purchase Amount (USD)', 'Location', 'Subscription Status', 'Frequency of Purchases']]

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
X_scaled_class = scaler.fit_transform(X_class)

# Thực hiện K-Means Clustering
kmeans = KMeans(n_clusters=2, random_state=42)  # Chia thành 2 nhóm: tiềm năng và không tiềm năng
df['Segment'] = kmeans.fit_predict(X_scaled_class)

# Tính khoảng cách Euclidean từ mỗi khách hàng đến các trung tâm cụm
centroids = kmeans.cluster_centers_
distances = np.linalg.norm(X_scaled_class - centroids[kmeans.labels_], axis=1)

# Thêm khoảng cách vào DataFrame
df['Distance to Centroid'] = distances

# Xác định ngưỡng (threshold) cho khoảng cách
threshold = np.median(distances)  # hoặc bạn có thể chọn giá trị khác tùy ý

# Phân loại khách hàng
df['Potential Customer'] = np.where(df['Distance to Centroid'] < threshold, 'Tiềm năng', 'Không tiềm năng')

In [11]:
# Giả sử df đã được định nghĩa và chứa dữ liệu cần thiết
X_potential = df[['Age', 'Gender', 'Purchase Amount (USD)', 'Location', 'Subscription Status', 'Frequency of Purchases']]
Y_potential = df['Potential Customer']

# Mã hóa Y_potential
label_encoders['Potential Customer'] = LabelEncoder()
Y_potential_encoded = label_encoders['Potential Customer'].fit_transform(Y_potential)

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
X_potential_scaled = scaler.fit_transform(X_potential)

# Huấn luyện mô hình
model_potential = RandomForestClassifier(random_state=42)
model_potential.fit(X_potential_scaled, Y_potential_encoded)
def predict_potential_customer(age, gender, purchase_amount, location, subscription_status, frequency_of_purchases):
    # Tạo DataFrame cho khách hàng mới
    new_customer_data = pd.DataFrame({
        'Age': [age],
        'Gender': [gender],
        'Purchase Amount (USD)': [purchase_amount],
        'Location': [location],
        'Subscription Status': [subscription_status],
        'Frequency of Purchases': [frequency_of_purchases]
    })

    # Mã hóa dữ liệu mới cho các cột cần thiết
    for col in ['Age','Gender', 'Location', 'Subscription Status', 'Frequency of Purchases']:
        if col in label_encoders:
            new_customer_data[col] = label_encoders[col].transform(new_customer_data[col])
        else:
            print(f"Không tìm thấy label encoder cho cột: {col}")

    # Đảm bảo rằng new_customer_data chỉ chứa các cột cần thiết
    new_customer_data = new_customer_data[['Age', 'Gender', 'Purchase Amount (USD)', 'Location', 'Subscription Status', 'Frequency of Purchases']]

    new_customer_data_scaled = scaler.transform(new_customer_data)

    # Dự đoán tiềm năng
    predicted_potential = model_potential.predict(new_customer_data_scaled)


    predicted_potential_name = label_encoders['Potential Customer'].inverse_transform(predicted_potential)

    # Hiển thị kết quả dự đoán
    print("Dự đoán tiềm năng khách hàng:", predicted_potential_name[0])

In [12]:
predict_potential_customer(44, 'Female', 77, 'Minnesota', "No",'Weekly')


Dự đoán tiềm năng khách hàng: Không tiềm năng


In [13]:
import joblib

# Chuẩn hóa dữ liệu cho xu hướng thời trang
scaler_trend = MinMaxScaler()
X_trend_scaled = scaler_trend.fit_transform(X_trend)

# Chuẩn hóa dữ liệu cho phân khúc khách hàng
scaler_potential = MinMaxScaler()
X_potential_scaled = scaler_potential.fit_transform(X_potential)

# Lưu scaler riêng cho từng mô hình
joblib.dump(scaler_trend, "models/scaler_trend.pkl")
joblib.dump(scaler_potential, "models/scaler_potential.pkl")


['models/scaler_potential.pkl']

In [14]:
import joblib


In [15]:
# Lưu model dự đoán xu hướng
joblib.dump(model_trend, "models/model_trend.pkl")

# Lưu model dự đoán phân khúc
joblib.dump(model_potential, "models/model_potential.pkl")

# Lưu scaler riêng cho từng mô hình
joblib.dump(scaler_trend, "models/scaler_trend.pkl")
joblib.dump(scaler_potential, "models/scaler_potential.pkl")

# Lưu label encoders
joblib.dump(label_encoders, "models/label_encoders.pkl")


['models/label_encoders.pkl']

In [16]:
scaler_trend = joblib.load("models/scaler_trend.pkl")
scaler_potential = joblib.load("models/scaler_potential.pkl")
